# Forum comments ETL

In [1]:
import json, os, datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

%matplotlib inline
plt.style.use('seaborn-whitegrid')

#data folders paths
raw_data_path = os.path.join('..', '..', 'Forum', 'data')
raw_data_path2 = os.path.join('..', '..', 'Forum', 'data2')
raw_data_path3 = os.path.join('..', '..', 'Forum', 'data3')
raw_data_path4 = os.path.join('..', '..', 'Forum', 'data4')
raw_data_path5 = os.path.join('..', '..', 'Forum', 'data5')
raw_data_path6 = os.path.join('..', '..', 'Forum', 'data6') //2019-01-15
data_path = os.path.join('..', 'data')
topics_path = os.path.join('..', 'topics', 'topic.pickle')

### Read raw crawled data files

In [4]:
file_names = [os.path.join(raw_data_path6, pos_json) for pos_json in os.listdir(raw_data_path6) if pos_json.endswith('.json')]

In [ ]:
file_names

In [5]:
%%time 

forum = list()
for i in range(len(file_names)):
    with open(file_names[i] ,encoding="utf-8") as json_data:
        forum += json.load(json_data)['posts']

Wall time: 888 ms


In [ ]:
#with open( os.path.join('..', '..','bitcoin.json') ,encoding="utf-8") as json_data:
#    forum = json.load(json_data)['posts']

In [7]:
def clean_string(text):
    return text.replace(u"\xad", '').replace(u"\n", ' ').replace(u"\\", '').replace(u"\t", ' ').strip()

In [8]:
%%time

comments = [(datetime.datetime.strptime(rep["date"][:10], '%Y-%m-%d').date(),
             clean_string(rep["content"]),
             clean_string(dic["topic"]),
             clean_string(dic["content"]),
            datetime.datetime.strptime(dic["date"][:10], '%Y-%m-%d').date(),
            dic["views"])
               for dic in forum  if 'replies' in dic 
               for rep in dic['replies'] if rep["content"] != []]

Wall time: 2.36 s


In [9]:
%%time
df = pd.DataFrame(comments, columns = ["Date", "Comment", "Topic", "Content", "Topic date", "Views"])
df = df.sort_values(["Date", "Topic"])

Wall time: 109 ms


In [10]:
#first crawl 736224
#second crawl 228781
#third crawl 312244
#bitcoin.json 627122
#appended first and bitcoin 1006057
#appended first second and bitcoin 1172448
#appended all with cleaned data 1094105
#data5 117314
len(df)

77264

In [11]:
if os.path.isfile(topics_path):
    topics = pd.read_pickle(topics_path)
else:
    topics = pd.DataFrame(columns= ["Topic", "Content", "Topic date", "Views"])

In [12]:
%%time
#insert new topics
indexes = []
i = 0
for topic in df["Topic"]:
    index = topics[topics["Topic"] == topic].index
    if len(index) == 0:
        topics = topics.append(df.iloc[[i]][["Topic", "Content", "Topic date", "Views"]], ignore_index= True)
        index = len(topics) - 1
    else:
        index = index[0]
    i += 1
    indexes.append(index)
df["Topic No."] = indexes
topics.to_pickle(topics_path)

Wall time: 3min 54s


In [13]:
%%time
#write processed data to pickle 
for date in df["Date"].unique():
    df2 = df[df["Date"] == date][["Topic No.", "Comment", "Date"]]
    if os.path.isfile(os.path.join(data_path, str(date))):
          df3 = pd.read_pickle(os.path.join(data_path, str(date)))
          df2 = pd.concat([df2, df3], ignore_index=True).drop_duplicates()
    df2.to_pickle(os.path.join(data_path, str(date)))

Wall time: 15.3 s


In [ ]:
%%time
#read pickle files
df = pd.DataFrame()
for file in os.listdir(data_path):
    help_df = pd.read_pickle(os.path.join(data_path, str(file)))
    help_df["Date"] = datetime.datetime.strptime(file, '%Y-%m-%d').date()
    df = pd.concat([df, help_df], ignore_index=True)

In [ ]:
os.system("start C:/Users/Giedrius/Music/1.mp3") 

In [ ]:
for i in range(len(topics)):
    index = df[df["Topic No."] == i].index[0]
    df.at[index, "Comment"] = topics.at[i,"Content"] + df.at[index, "Comment"]

In [ ]:
a = df.groupby(["Date", "Topic", "Content"])["Comment"].apply(lambda x: ''.join(x))

In [ ]:
a.index[50000]


In [ ]:
df2 = df[["Comment","Date"]].groupby(["Date"]).count()

In [ ]:
df2

In [ ]:
df2.plot()

In [ ]:
ax = df2["Comment"][:-1].plot(figsize=(8,6), x = "Data")
ax.set_xlabel("Data",fontsize=13)
ax.set_ylabel("Komentarų skaičius", fontsize=13)


In [ ]:
df[-1:]

In [ ]:
df2 = df.groupby(["Date"]).count()
# Set up the axes and figure
fig, ax = plt.subplots(figsize=(20,10))
ax.bar(df2.index, df2.Comment, align='center')
ax.xaxis_date()
fig.autofmt_xdate()
plt.show()
